In [1]:
import pandas as pd
import numpy as np
from libs import have_dream
from libs import constants as cst
from IPython.display import display

#T-38 parameters
Sref=170*cst.FT_TO_M**2
AR=3.75
confs=['Clean','SB','Shuttle','X-24']


## Sawtooth descent 3D

In [ ]:

full_df = pd.read_csv(r"data\Have dream\Master sawtooth.csv", sep=',')



summary_rows = []

for conf in confs:
    # valid_df = full_df[(full_df['Conf'] == conf)]
    valid_df = full_df[(full_df['Conf'] == conf) & (full_df['valid'])]
    df = full_df[(full_df['Conf'] == conf)]

    if df.empty:
        print(f"⚠️ No data for configuration {conf}")
        continue

    # Fit drag models
    results = have_dream.fit_drag_models(valid_df, Sref)

    coeffs2 = results["2coeff"]["coeffs"]
    coeffs3 = results["3coeff"]["coeffs"]

    # Compute max L/D vs weight
    ld_table = have_dream.ld_max_vs_weight(df, coeffs2, Sref, step=500)
    print(f"\n--- Max L/D vs Weight for {conf} ---")
    print(ld_table.to_string(index=False))

    # Compute RMSEs on measured data
    V_ms = valid_df["TAS"].values * cst.KT_TO_MS
    W_kg = valid_df["weight_lb"].values * cst.LB_TO_KG
    rho = valid_df["density"].values
    L_over_D = valid_df["l_over_d"].values

    pred2 = have_dream.ld_2coeff(V_ms, W_kg, rho, Sref, *coeffs2)
    pred3 = have_dream.ld_3coeff(V_ms, W_kg, rho, Sref, *coeffs3)

    rmse2 = np.sqrt(np.mean((L_over_D - pred2) ** 2))
    rmse3 = np.sqrt(np.mean((L_over_D - pred3) ** 2))

    # Plot both surfaces with RMSE
    have_dream.plot_ld_surface_models(valid_df, coeffs2, coeffs3, Sref)

    # Save to summary table
    summary_rows.append({
        "Configuration": conf,
        "Cd0 (2c)": coeffs2[0],
        "k (2c)": coeffs2[1],
        "RMSE (2c)": rmse2,
        "Cd0 (3c)": coeffs3[0],
        "k (3c)": coeffs3[1],
        "kl (3c)": coeffs3[2],
        "RMSE (3c)": rmse3,
    })

# Build summary DataFrame
summary_df = pd.DataFrame(summary_rows).round(4)

display(summary_df)




--- Max L/D vs Weight for Clean ---
 Weight_lb    Max_LD     TAS_kt
     10000 10.379129 290.813768
     10500 10.379155 298.349585
     11000 10.379144 305.131821
     11500 10.379136 311.914056
     12000 10.379144 318.696291
     12500 10.379155 325.478527



--- Max L/D vs Weight for SB ---
 Weight_lb   Max_LD     TAS_kt
     10000 6.270211 238.019742
     10500 6.270209 243.998459
     11000 6.270207 249.478949
     11500 6.270202 255.457666
     12000 6.270203 260.938156
     12500 6.270205 265.920420



--- Max L/D vs Weight for Shuttle ---
 Weight_lb   Max_LD     TAS_kt
     10000 4.434963 219.731020
     10500 4.434961 225.011748
     11000 4.434961 230.292475
     11500 4.434963 235.573203
     12000 4.434958 240.853930
     12500 4.434962 245.547910



--- Max L/D vs Weight for X-24 ---
 Weight_lb   Max_LD     TAS_kt
     10000 4.778484 213.804871
     10500 4.788749 213.804871
     11000 4.789670 216.642797
     11500 4.789664 221.751063


,Configuration,Cd0 (2c),k (2c),RMSE (2c),Cd0 (3c),k (3c),kl (3c),RMSE (3c)
0,Clean,0.0135,0.1720,0.9423,0.0301,0.3325,-0.1073,0.8335
1,SB,0.0331,0.1921,0.4058,0.0605,0.3726,-0.1439,0.3875
2,Shuttle,0.0548,0.2320,0.1941,0.0900,0.4420,-0.1779,0.1683
3,X-24,0.0574,0.1899,0.3199,0.1467,0.7480,-0.4668,0.2057


## Incremental drag descent